In [ ]:
import kagglehub


path = kagglehub.dataset_download("kangjiayong/deepfake-detection")

print("Path to dataset files:", path)

100%|██████████| 23.4G/23.4G [05:53<00:00, 71.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kangjiayong/deepfake-detection/versions/1


In [ ]:
!ls /root/.cache/kagglehub/datasets/kangjiayong/deepfake-detection/versions/1/

FAKE  REAL  selected_video.csv	shape_predictor_68_face_landmarks.dat


In [ ]:
from typing import List, Tuple, Dict, Union
from collections import defaultdict
import os
import cv2
from typing import List, Tuple, Dict, Union
import pandas as pd
from tqdm import tqdm


def convert_path(path: str) -> str:
    for i in range(10):
        path = path.replace(f'{i:02}', str(i))
    return path


def get_video_info(video_path: str) -> Union[Tuple[int, float], Tuple[None, None]]:
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Не вдалося відкрити файл: {video_path}")
        return None, None

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    return frame_count, fps


def process_file_paths(df: pd.DataFrame) -> List[Tuple[str, str]]:
    result: List[Tuple[str, str]] = []

    for _, row in df.iterrows():
        file_name = row['path']

        path_parts = file_name.split('\\')
        curr_path = path_parts[-1].replace('.zip', '')
        file_path = os.path.join('/root/.cache/kagglehub/datasets/kangjiayong/deepfake-detection/versions/1/', row['label'], row['label'], convert_path(curr_path), row['file_name'])
        file_path_target = os.path.join('/kaggle/working/deepfake-detection', row['label'], row['label'], convert_path(curr_path))

        result.append((file_path, row['label']))

    return result


df = pd.read_csv('/root/.cache/kagglehub/datasets/kangjiayong/deepfake-detection/versions/1/selected_video.csv')

video_stats = []
for video_path, label in tqdm(process_file_paths(df)):
    frame_count, fps = get_video_info(video_path)
    if frame_count is not None and fps is not None:
        video_stats.append({
            'file_path': video_path,
            'label': label,
            'frames': frame_count,
            'fps': fps
        })

df = pd.DataFrame(video_stats)

stats_fps = df['fps'].describe()
stats_frames = df['frames'].describe()

print("FPS статистика:")
print(stats_fps)

print("\nКількість кадрів статистика:")
print(stats_frames)

100%|██████████| 5024/5024 [01:47<00:00, 46.72it/s]

FPS статистика:
count    5024.000000
mean       29.462446
std         1.801130
min        12.473600
25%        29.964500
50%        29.970000
75%        29.970000
max        30.079600
Name: fps, dtype: float64

Кількість кадрів статистика:
count    5024.000000
mean      294.985669
std        17.884431
min       125.000000
25%       300.000000
50%       300.000000
75%       300.000000
max       301.000000
Name: frames, dtype: float64
